In [3]:
import math
import torch
import torch.nn as nn

# TRANSFORMER WITH LORA and QLORA

In [33]:
class LORALayer(nn.Module):
    def __init__(self, input_dim, output_dim, rank, alpha=1):
        super(LORALayer, self).__init__()
        self.rank = rank
        self.alpha = alpha

        # Original weight and bias of the linear layer
        self.weight = nn.Parameter(torch.Tensor(output_dim, input_dim))
        print("self.weight Shape:", self.weight.shape)
        self.bias = nn.Parameter(torch.Tensor(output_dim))

        # LORA specific parameters
        self.A = nn.Parameter(torch.Tensor(input_dim, rank))
        self.B = nn.Parameter(torch.Tensor(rank, output_dim))

        self.reset_parameters()

    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        nn.init.zeros_(self.bias)
        nn.init.normal_(self.A, 0, 0.02)
        nn.init.normal_(self.B, 0, 0.02)

    def forward(self, x):
        print("LORALayer Input Shape:", x.shape)
        original_size = x.size()
        x = x.reshape(-1, original_size[-1])

        # Compute lora_adjustment
        lora_adjustment = self.alpha * (x @ self.A) @ self.B
        # Assuming each element in the batch gets the same adjustment
        lora_adjustment = lora_adjustment.mean(dim=0)

        print("Adjusted lora_adjustment Shape:", lora_adjustment.shape)
        print("self.weight Shape:", self.weight.shape)
        
        # Ensure lora_adjustment is broadcastable to the weight shape
        lora_adjustment = lora_adjustment.unsqueeze(0).expand_as(self.weight)

        x = nn.functional.linear(x, self.weight + lora_adjustment, self.bias)
        print("LORALayer Output Shape:", x.shape)

        return x.reshape(original_size)

class QLORALayer(nn.Module):
    def __init__(self, input_dim, output_dim, rank, alpha=1, quantization_bits=8):
        super(QLORALayer, self).__init__()
        self.rank = rank
        self.alpha = alpha
        self.quantization_bits = quantization_bits

        # Original weight and bias
        self.weight = nn.Parameter(torch.Tensor(output_dim, input_dim))
        self.bias = nn.Parameter(torch.Tensor(output_dim))

        # QLORA specific parameters
        self.A = nn.Parameter(torch.Tensor(input_dim, rank))
        self.B = nn.Parameter(torch.Tensor(rank, output_dim))

        self.reset_parameters()

    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        nn.init.zeros_(self.bias)
        nn.init.normal_(self.A, 0, 0.02)
        nn.init.normal_(self.B, 0, 0.02)

    def quantize(self, x, num_bits):
        # Implement a simple quantization method
        scale = x.abs().max()
        x_quantized = torch.round(x / scale * (2**num_bits - 1))
        return x_quantized, scale

    def forward(self, x):
        print("QLORALayer Input Shape:", x.shape)
        original_size = x.size()
        x = x.reshape(-1, original_size[-1])

        A_quantized, scale_A = self.quantize(self.A, self.quantization_bits)
        B_quantized, scale_B = self.quantize(self.B, self.quantization_bits)

        lora_adjustment = self.alpha * (x @ (A_quantized / scale_A)) @ (B_quantized / scale_B)
        lora_adjustment = lora_adjustment.mean(dim=0)

        print("Adjusted lora_adjustment Shape:", lora_adjustment.shape)
        print("self.weight Shape:", self.weight.shape)

        # Ensure lora_adjustment is broadcastable to the weight shape
        lora_adjustment = lora_adjustment.unsqueeze(0).expand_as(self.weight)

        x = nn.functional.linear(x, self.weight + lora_adjustment, self.bias)
        print("QLORALayer Output Shape:", x.shape)

        return x.reshape(original_size)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)


class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1, lora_alpha=2, qlora_alpha=2, rank=16, quantization_bits=8):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)

        # Use LORALayer and QLORALayer in place of linear layers
        self.linear1 = LORALayer(d_model, dim_feedforward, rank, lora_alpha)  # d_model -> dim_feedforward
        self.linear2 = QLORALayer(dim_feedforward, d_model, rank, qlora_alpha, quantization_bits)  # dim_feedforward -> d_model

        self.activation = nn.ReLU()
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src, src_mask=None, src_key_padding_mask=None, is_causal=False):
        print("Encoder Layer Input Shape:", src.shape)
        src2 = self.self_attn(src, src, src, attn_mask=src_mask, key_padding_mask=src_key_padding_mask)[0]
        src = src + self.dropout1(src2)
        src = self.norm1(src)

        src2 = self.linear2(self.dropout(self.activation(self.linear1(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        print("Encoder Layer Output Shape:", src.shape)
        return src


class TransformerModel(nn.Module):
    def __init__(self, ntoken, d_model, nhead, nlayers, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask, src_key_padding_mask)
        output = self.decoder(output)
        return output

# Example usage
ntoken = 1000  # size of vocabulary
d_model = 2048  # embedding dimension
nhead = 8      # number of heads in nn.MultiheadAttention
nlayers = 2    # number of TransformerEncoderLayer
dropout = 0.2  # dropout probability

model = TransformerModel(ntoken, d_model, nhead, nlayers, dropout)

# Create dummy input data
seq_length = 10
dummy_input = torch.randint(ntoken, (seq_length, 1))  # Batch size of 1

# Forward pass
output = model(dummy_input)

# Check output shape
expected_output_shape = (seq_length, 1, ntoken)
assert output.shape == expected_output_shape, f"Output shape is incorrect: expected {expected_output_shape}, got {output.shape}"
print("Basic forward pass test passed.")

# Set up loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Create dummy input and target data
seq_length = 10
dummy_input = torch.randint(ntoken, (seq_length, 1))  # Batch size of 1
dummy_target = torch.randint(ntoken, (seq_length, 1))  # Dummy target

# Save initial state of a specific weight for comparison
initial_weight = model.encoder.weight.data.clone()

# Forward pass
output = model(dummy_input)
loss = criterion(output.view(-1, ntoken), dummy_target.view(-1))

# Perform backward pass
optimizer.zero_grad()
loss.backward()
optimizer.step()

# Check if the weights have been updated
assert not torch.equal(initial_weight, model.encoder.weight.data), "Weights did not update after optimization step"
print("Backward pass test passed. Weights updated.")


self.weight Shape: torch.Size([2048, 2048])
Encoder Layer Input Shape: torch.Size([10, 1, 2048])
LORALayer Input Shape: torch.Size([10, 1, 2048])
Adjusted lora_adjustment Shape: torch.Size([2048])
self.weight Shape: torch.Size([2048, 2048])
LORALayer Output Shape: torch.Size([10, 2048])
QLORALayer Input Shape: torch.Size([10, 1, 2048])
Adjusted lora_adjustment Shape: torch.Size([2048])
self.weight Shape: torch.Size([2048, 2048])
QLORALayer Output Shape: torch.Size([10, 2048])
Encoder Layer Output Shape: torch.Size([10, 1, 2048])
Encoder Layer Input Shape: torch.Size([10, 1, 2048])
LORALayer Input Shape: torch.Size([10, 1, 2048])
Adjusted lora_adjustment Shape: torch.Size([2048])
self.weight Shape: torch.Size([2048, 2048])
LORALayer Output Shape: torch.Size([10, 2048])
QLORALayer Input Shape: torch.Size([10, 1, 2048])
Adjusted lora_adjustment Shape: torch.Size([2048])
self.weight Shape: torch.Size([2048, 2048])
QLORALayer Output Shape: torch.Size([10, 2048])
Encoder Layer Output Shape: 

# Define Multi Head Attention / Transformer / Block


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(MultiHeadAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        # Einsum does the matrix multiplication for query*keys for each training example
        # with every other training example, then sum it up
        attention = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

        if mask is not None:
            attention = attention.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(attention / (self.embed_size ** (1 / 2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )

        out = self.fc_out(out)
        return out

class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = MultiHeadAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        # Add skip connection, run through a normalization layer, and finally a dropout layer
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out
    
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, embed_size=256, num_layers=6, forward_expansion=4, heads=8, dropout=0, device="cuda", max_length=100):
        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length,
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length,
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )

        return trg_mask.to(self.device)

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out


class Encoder(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length,
    ):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)

        out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out

class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.attention = MultiHeadAttention(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out

class Decoder(nn.Module):
    def __init__(
        self, 
        trg_vocab_size, 
        embed_size, 
        num_layers, 
        heads, 
        forward_expansion, 
        dropout, 
        device, 
        max_length
    ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(num_layers)
            ]
        )

        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)

        return out

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Transformer(
    src_vocab_size=NUM_WORDS_SRC,
    trg_vocab_size=NUM_WORDS_TRG,
    src_pad_idx=PAD_IDX_SRC,
    trg_pad_idx=PAD_IDX_TRG,
    device=device
).to(device)
